Polygenic Risk Score: Workshop Exercise with Coronary Artery Disease

Code adapted from MiCM GWAS & PRS workshop by Zoe Schmilovich, PhD (2024 June)

# Install tools



STEP 1. Install commandline tool for retrieving data from Google Drive

In [ ]:
!pip install gdown

STEP 2. Install plink2

In [ ]:
!wget https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_20250806.zip
!unzip plink2_linux_x86_64_20250806.zip
!mv plink2 /usr/local/bin/

# Visualization summary statistics of CAD GWAS

STEP 3. Download summary statistics of GWAS of CAD from van der Herst et al. (2018)

In [ ]:
# acquire summary statistics from Google drive
# file name: raw_cad_gwas_sum_stats
!gdown --id 1tgLX51-EShA4ZwroxTbjoNhbvsxc2V-y

STEP 4.Visualize summary statistics with Manhattan plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# CAD GWAS sumstats we just downloaded
input_file = "raw_cad_gwas_sum_stats"

# Load sumstats (tab separated file)
df = pd.read_csv(input_file, delim_whitespace=True)

# Data cleaning: keep only autosomes (1-22) and drop NA pvals
df = df[df['chr'].astype(str).isin([str(i) for i in range(1, 23)])]
df = df.dropna(subset=['pval'])

# Convert chromosome and basepair positions to numeric, p-value to float
df['chr'] = df['chr'].astype(int)
df['bp'] = df['bp'].astype(int)
df['pval'] = df['pval'].astype(float)

# Sort by chromosome and base pair
df = df.sort_values(['chr', 'bp'])

# Compute -log10(p), this will be the Y-axis
df['-log10p'] = -np.log10(df['pval'])

# Cumulative basepairs
cumulative_bp = 0
ticks = []
labels = []
for chrom in sorted(df['chr'].unique()):

    # create a chromosome mask (true/false)
    chr_mask = df['chr'] == chrom

    # create a new column for cumulative position for visualization
    # for each chromosome, set cum_bp as to 0 (by substracting actual basepair value by min basepair)
    # then add cumulative basepair to reflect global position
    df.loc[chr_mask, 'cum_bp'] = df.loc[chr_mask, 'bp'] - df.loc[chr_mask, 'bp'].min() + cumulative_bp

    # place the chromosome number at the middle of each chromosome block
    # then save the chromosome number as a string for labeling the x axis
    ticks.append(df.loc[chr_mask, 'cum_bp'].median())
    labels.append(str(chrom))

    # Add a fixed gap between chromosomes for visibility
    cumulative_bp += (df.loc[chr_mask, 'bp'].max() - df.loc[chr_mask, 'bp'].min()) + 1e6


# Assign fixed color per chromosome
chromosomes = sorted(df['chr'].unique())
cmap = cm.get_cmap('tab20', len(chromosomes))
chrom_colors = {chrom: cmap(i) for i, chrom in enumerate(chromosomes)}

# Plot
plt.figure(figsize=(18, 6))
for chrom in chromosomes:
    chr_data = df[df['chr'] == chrom]
    plt.scatter(chr_data['cum_bp'], chr_data['-log10p'],
                color=chrom_colors[chrom], s=6)

# Significance line
plt.axhline(y=-np.log10(5e-8), color='red', linestyle='--', linewidth=1)

plt.xticks(ticks, labels)
plt.xlabel('Chromosome')
plt.ylabel('-log10(p-value)')
plt.title('Manhattan plot of CAD GWAS')
plt.tight_layout()
plt.show()

STEP 5. Visualize SNPs by p-value

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CAD GWAS sumstats we just downloaded
input_file = "raw_cad_gwas_sum_stats"

# Load sumstats (tab separated file)
df = pd.read_csv(input_file, delim_whitespace=True)

# Define the thresholds
thresholds = [1e-4, 1e-5, 1e-6, 5e-8]

# Count the number of SNPs below each threshold
counts = [np.sum(df['pval'] < threshold) for threshold in thresholds]

# Create the bar plot
plt.figure(figsize=(10, 6))
plt.bar([str(thresh) for thresh in thresholds], counts, color='blue')

# Add labels and title
plt.xlabel('P-value Thresholds')
plt.ylabel('Number of SNPs')
plt.title('Number of SNPs at Different P-value Thresholds')
plt.legend()

# Show the plot
plt.show()

# Preprocessing of Individual Genotyping Data

STEP 1. Download the genotype data of a portion of European participants from 1000 Genomes Consortium


Binary files contain genotyped data of 44,000 genetic variants and 628 participants. These individuals do not contain more than 1% missing genotyping

Data processed by Kevin Liang, PhD

In [ ]:
# EUR contains plink binary files (bed bim fam)
!gdown 1s_5XroEhHM5uc_brlZGKZbmwPutM6Td-
!unzip EUR.zip

STEP 2. Download mock (simulated) phenotype data

In [ ]:
# CAD_phenotypes.txt contains ID and phenotype (0 or 1)
!gdown 1cclXWTnzkbxAoYwmiC1phihS3seDBTyp

STEP 3. Integrate genotype with phenotype




*   ``` --bfile``` reads binary files of genotype data
*   ``` --pheno``` specifies phenotype of participants
*   ``` --make-bed``` tells plink to output binary files
*   ``` --out``` specifies output directory and file name


In [ ]:
# this generates binary files with phenotype integrated
!plink2 --bfile EUR --pheno CAD_phenotypes.txt --make-bed --out CAD_case_control

Visualize minor allele frequency (MAF) of all SNPs in 628 individuals
*   ```--freq``` tells plink to output the frequency of the alleles

In [ ]:
# this generates an allele frequency file (.afreq) for all SNPs
!plink2 --bfile CAD_case_control --freq --out freq_counts

Load freq_count.afreq into python

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the .afreq file into a pandas DataFrame
afreq_df = pd.read_csv("freq_counts.afreq", delim_whitespace=True)

# Display the first few rows of the DataFrame
print(afreq_df.head())

plt.figure(figsize=(10, 6))
plt.hist(afreq_df['ALT_FREQS'], bins=50, edgecolor='k', alpha=0.7)
plt.axvline(x=0.01, color='red', linestyle='--', linewidth=1.5, label='MAF 0.01')

plt.xlabel('Alternative Allele Frequency')
plt.ylabel('Number of Variants')
plt.title('Distribution of Alternative Allele Frequencies')
plt.grid(True)
plt.legend()
plt.show()

STEP 4. Exclude SNPs with extremely low frequency, SNPs with low genotyping rate and SNPSs that violate the Hardy-Weinberg equilibrium



*   ```--maf 0.01``` excludes SNPs that occurs in less than 1% of population
*   ```--geno 0.1``` excludes SNPs with more than 10% genotype rate
*   ```---hwe 0.0001``` excludes SNPs that violate H-W equilibirium, less than Chisq P-value threshold of 0.0001



In [ ]:
# this generates thresholded binary files
!plink2 --bfile CAD_case_control --maf 0.01 --geno 0.1 --hwe 0.0001 --make-bed --out CAD_maf-0.01_geno-0.1_hwe-1e4

STEP 5a. Run LD pruning, to ensure independence of SNPs when scoring PRS.  


* ```--indep-pairwise <window> <step-size> <LD threshold> ``` removes SNPs that are correlated, with a stepwise sliding window at the specified threshold (we will use Rsq > 0.2)



In [ ]:
# this generates .in and .out file, .in file contains variants that are independent
!plink2 --bfile CAD_maf-0.01_geno-0.1_hwe-1e4 --indep-pairwise 50 5 0.2 --out CAD_ld_pruned

STEP 5b. Create binary files from list of SNPs that are LD pruned (independent from each other)
*  ```--extract``` tells plink to use a certain set of SNPs

In [ ]:
# this generates binary files from the .in file of LD pruning
!plink2 --bfile CAD_maf-0.01_geno-0.1_hwe-1e4 --extract CAD_ld_pruned.prune.in --make-bed --out CAD_EUR_QCed

# Use PRS-CS to calculate weight of each SNP in PRS scoring

 Since PRS-CS is computationally intensive, we will use pre-downloaded PRSice weights of CAD for downstream analysis.

In [ ]:
# Clone the PRS-CS repository
!git clone https://github.com/getian107/PRScs.git

# Navigate to the PRS-CS directory
%cd PRScs

# Download the pre-trained LD reference panels
!wget https://personal.broadinstitute.org/hhuang//public/PRS-CSx/Reference/1KG/ldblk_1kg_eur.tar.gz

# Extract the downloaded files
!tar -xzvf ldblk_1kg_eur.tar.gz

# Compute PRS-CS to provide accurate SNP effect sizes ahead of individual-level PRS calculation
!python PRScs.py \
        --ref_dir=/PRScs/ldblk_1kg_eur \
        --bim_prefix=[prefix of the QCed PLINK file] \
        --sst_file=[GWAS summary statistics file] \
        --n_gwas=[# of individuals included in GWAS] \
        --out_dir=[path]/[output_prefix]

# PRS-CS will output the SNP effect sizes for each chromosome separately
# To concatenate all chromosome files:
cat [output_prefix]_pst_eff_a1_b0.5_phiauto_chr*.txt | sort -n -k1 > [output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.txt

# Prepare the output of PRScs so that it can be used by PLINK
# We will use this output to generate individual-level scores for our samples an allelic scoring system involving one or more SNPs.

awk '{print $2, $4, $6}' \
[output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.txt | \
sed 's/ /\t/g' > [output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore

This generates the adjusted weights of SNPs. We will download the pre-computed weights instead from Google Drive

In [ ]:
# this will generate a file named CAD_EUR_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore
!gdown --id 1X83iPlOSEAkrBvEwL1ab4m9FCj37I4_y

# Scoring individuals with PRS using PLINK

The adjusted weights file contains how much each SNP contributes to CAD risk. We will score all 628 individuals using these weights.

*   ```--score``` tells plink the PRS weights

In [ ]:
# this generates an .sscore file of a score for each individual, calculated from their genotype
!plink2 \
  --bfile CAD_EUR_QCed \
  --pheno CAD_phenotypes.txt \
  --score CAD_EUR_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore \
  --out prs_for_cad

Visualize how the PRS performed.

We will perform a t-test to see if PRS can distinguish between cases and controls

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler

# Load the data into a DataFrame
file_path = "prs_for_cad.sscore"  # Update the path as needed
df = pd.read_csv(file_path, delim_whitespace=True)

# Ensure the Phenotype is binary (0 for control, 1 for case)
df['Phenotype'] = df['Phenotype'].apply(lambda x: 1 if x == 2 else 0).astype(int)

# Standardize the SCORE1_AVG values
scaler = StandardScaler()
df['SCORE1_AVG_SCALED'] = scaler.fit_transform(df[['SCORE1_AVG']])

# Split the data into cases and controls
cases = df[df['Phenotype'] == 1]['SCORE1_AVG_SCALED']
controls = df[df['Phenotype'] == 0]['SCORE1_AVG_SCALED']

# Calculate the mean SCORE1_AVG for cases and controls
mean_cases = df[df['Phenotype'] == 1]['SCORE1_AVG_SCALED'].mean()
mean_controls = df[df['Phenotype'] == 0]['SCORE1_AVG_SCALED'].mean()

print(f"Mean SCORE1_AVG for cases: {mean_cases}")
print(f"Mean SCORE1_AVG for controls: {mean_controls}")

# Perform t-test
t_stat, p_val = ttest_ind(cases, controls)
print(f"t-statistic: {t_stat}, p-value: {p_val}")

# Set the plot style
sns.set(style="white")

# Create a box plot to view the distribution of PRS
plt.figure(figsize=(10, 6))
ax = sns.boxplot(x="Phenotype", y="SCORE1_AVG_SCALED", data=df, hue="Phenotype", palette="Set2", legend=False)

# Add annotation for the p-value
x1, x2 = 0, 1  # x-coordinates for controls and cases
y, h, col = df['SCORE1_AVG_SCALED'].max() + 0.1, 0.05, 'k'
plt.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, color=col)
plt.text((x1 + x2) * .5, y + h, f"p = {p_val:.3e}", ha='center', va='bottom', color=col)

plt.title('Distribution of scaled polygenic risk for CAD')
plt.xlabel('CAD diagnosis')
plt.ylabel('Polygenic risk for CAD')
plt.xticks([0, 1], ['Control', 'Case'])
plt.show()

PRS does not distinguish between cases and controls. This is most likely due to the fact that we used simulated dat and not real disease status.